In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.reset_option('display.float_format', silent=True)
nft_trades_clustered = pd.read_csv('nft_traders_classified.csv')
nft_trades_clustered = nft_trades_clustered[(nft_trades_clustered['total_trade_count']>=20)&(nft_trades_clustered['avg_nb_minutes_between_trades']>=1000)].reset_index(drop=True)

In [3]:
nft_trades_clustered['avg_sol_per_trade'] = nft_trades_clustered['total_sol_amount_traded']/nft_trades_clustered['total_trade_count']
nft_trades_clustered['buy_to_total_trade_ratio'] = nft_trades_clustered['total_buys']/nft_trades_clustered['total_trade_count']*nft_trades_clustered['total_sol_amount_traded']

In [4]:
scaler = MinMaxScaler()

nft_trades_clustered['normalized_avg_nb_hours_between_hold_trade'] = scaler.fit_transform(nft_trades_clustered['avg_nb_hours_between_hold_trade'].values.reshape(-1, 1))
nft_trades_clustered['normalized_avg_sol_per_trade'] = scaler.fit_transform(nft_trades_clustered['avg_sol_per_trade'].values.reshape(-1, 1))
nft_trades_clustered['normalized_buy_to_total_trade_ratio'] = scaler.fit_transform(nft_trades_clustered['buy_to_total_trade_ratio'].values.reshape(-1, 1))
nft_trades_clustered['normalized_avg_nb_minutes_between_trades'] = scaler.fit_transform(nft_trades_clustered['avg_nb_minutes_between_trades'].values.reshape(-1, 1))
nft_trades_clustered['normalized_total_trade_count'] = scaler.fit_transform(nft_trades_clustered['total_trade_count'].values.reshape(-1, 1))
nft_trades_clustered['normalized_total_sol_traded'] = scaler.fit_transform(nft_trades_clustered['total_sol_amount_traded'].values.reshape(-1, 1))

In [5]:
weight_tier_1 = 9.5
weight_tier_2 = 9

weight_time_between_trades = 10

weight_avg_sol_trade = weight_tier_1
weight_avg_hours_hold = weight_tier_1
weight_total_trades = weight_tier_1
weight_total_sol = weight_tier_1

weight_buy_to_trade = weight_tier_2

nft_trades_clustered['combined_score'] = (nft_trades_clustered['normalized_avg_nb_minutes_between_trades'] * weight_time_between_trades)+ \
                                (nft_trades_clustered['normalized_avg_sol_per_trade'] * weight_avg_sol_trade) + \
                               (nft_trades_clustered['normalized_avg_nb_hours_between_hold_trade'] * weight_avg_hours_hold)+ \
                               (nft_trades_clustered['normalized_buy_to_total_trade_ratio'] * weight_buy_to_trade)+ \
                               (nft_trades_clustered['normalized_total_trade_count'] * weight_total_trades)+ \
                               (nft_trades_clustered['normalized_total_sol_traded'] * weight_total_sol)
                               
nft_trades_clustered['rank'] = nft_trades_clustered['combined_score'].rank(ascending=False)
nft_trades_clustered.sort_values('rank').reset_index(drop=True).to_csv('nft_traders_ranking.csv',index=False)
nft_trades_clustered.sort_values('rank').reset_index(drop=True).head(40)

,address,total_sol_amount_traded,total_trade_count,unique_nft_traded,total_buys,total_sells,nb_days_since_first_trade,avg_nb_minutes_between_trades,avg_nb_hours_between_hold_trade,Cluster,...,avg_sol_per_trade,buy_to_total_trade_ratio,normalized_avg_nb_hours_between_hold_trade,normalized_avg_sol_per_trade,normalized_buy_to_total_trade_ratio,normalized_avg_nb_minutes_between_trades,normalized_total_trade_count,normalized_total_sol_traded,combined_score,rank
0,FwHQiQ5m2nCFtmxWCYGh4GzfzFdzcvYXuwKkcJXBxrqr,23204.542897,31,28,27,4,291,13609.419355,146.733333,1,...,748.533642,20210.408330,0.161427,1.000000,1.000000,0.199903,0.009955,0.673140,28.521985,1.0
1,DD9eeertZsaHXzxiBwaBV9BSgMhPHb2yuvBH5ivuxAFV,34472.035440,531,367,287,244,414,1942.365348,705.663934,1,...,64.919087,18631.778101,0.196888,0.086723,0.921890,0.014935,0.462443,1.000000,25.033888,2.0
2,9PGjxWhx5iARoTz6gwDmDBStFoDq591nCHBRXqXcej2d,22415.435554,961,826,362,599,409,1243.693028,2671.699082,1,...,23.325115,8443.691645,0.321622,0.031156,0.417789,0.003859,0.851584,0.650248,21.417482,3.0
3,DNV6Sah7BCHCokT6b5GgphZVKsCPLMgRwNQ8Nq2dKm6D,27966.474891,473,331,253,220,414,1261.509514,248.002652,1,...,59.125740,14958.812151,0.167852,0.078983,0.740154,0.004141,0.409955,0.811279,20.649461,4.0
4,HJsgX8WaKGaLbJ243vN6edt87Qzd5p2Z7ZskeFvjt9oD,19804.810195,887,485,421,466,347,1053.069899,733.688698,1,...,22.327858,9400.028289,0.198666,0.029823,0.465108,0.000837,0.784615,0.574516,19.276744,5.0
5,8LaUZQHpxiUbWbSh2oBzqJ3yfEC66QHBbq8R4WbahUAR,15395.588613,917,695,663,254,381,1208.512541,941.498556,1,...,16.789082,11131.161669,0.211851,0.022424,0.550764,0.003301,0.811765,0.446609,19.170038,6.0
6,6XtgBQQ6Y77eWMwWPJKybfTbMsryTdaK9gPnJCCdT1Sr,21393.232537,53,32,24,29,274,7459.150943,1313.079885,1,...,403.645897,9687.501526,0.235425,0.539246,0.479332,0.102398,0.029864,0.620595,18.876710,7.0
7,HFE5NcQT5TKG2vYnPyCRxwLmG3UgL8Ayu8yCYko2gstk,22943.740965,556,254,269,287,407,1041.482014,368.110976,1,...,41.265721,11100.478992,0.175473,0.055123,0.549246,0.000653,0.485068,0.665574,18.071500,8.0
8,DJBHGNxosDcJb16u7Lgy83f9k5Seru1KMbzK16EX9qVR,246.181845,974,974,960,14,392,1145.520534,13364.210714,1,...,0.252753,242.643297,1.000000,0.000332,0.012006,0.002302,0.863348,0.007137,17.903844,9.0
9,GFexHU2EkUcmcKbj3GyiuNQoa7TreCxLkAxMb2xeLHgS,13923.837864,874,576,541,333,397,1050.558352,1355.893443,1,...,15.931165,8618.760051,0.238142,0.021278,0.426452,0.000797,0.772851,0.403914,17.489783,10.0


In [6]:
nft_trades_clustered.describe()

,total_sol_amount_traded,total_trade_count,unique_nft_traded,total_buys,total_sells,nb_days_since_first_trade,avg_nb_minutes_between_trades,avg_nb_hours_between_hold_trade,Cluster,AnomalyScore,avg_sol_per_trade,buy_to_total_trade_ratio,normalized_avg_nb_hours_between_hold_trade,normalized_avg_sol_per_trade,normalized_buy_to_total_trade_ratio,normalized_avg_nb_minutes_between_trades,normalized_total_trade_count,normalized_total_sol_traded,combined_score,rank
count,43271.000000,43271.000000,43271.000000,43271.000000,43271.000000,43271.000000,43271.000000,43271.000000,43271.0,43271.000000,43271.000000,43271.000000,43271.000000,43271.000000,43271.000000,43271.000000,43271.000000,43271.000000,43271.000000,43271.000000
mean,461.834944,116.388528,89.586143,61.867717,54.520811,340.922396,10177.879590,2065.827717,1.0,0.354517,4.298943,236.259153,0.283183,0.005737,0.011690,0.145500,0.087229,0.013393,5.260865,21636.000000
std,971.046851,120.618499,91.072427,74.552697,61.999733,65.768536,8704.974410,2141.342683,0.0,0.070745,8.198979,538.842524,0.135856,0.010953,0.026662,0.138007,0.109157,0.028169,1.932453,12491.406086
min,0.146317,20.000000,1.000000,0.000000,1.000000,77.000000,1000.291759,-2397.665833,1.0,0.073211,0.004303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.488603,1.000000
25%,54.288497,37.000000,30.000000,16.000000,16.000000,300.000000,3557.334790,420.789736,1.0,0.316587,0.885732,21.898266,0.178815,0.001178,0.001084,0.040539,0.015385,0.001571,3.827848,10818.500000
50%,166.297839,70.000000,55.000000,34.000000,31.000000,358.000000,7529.350515,1304.892803,1.0,0.374405,2.213338,76.373538,0.234906,0.002951,0.003779,0.103510,0.045249,0.004820,4.834806,21636.000000
75%,474.893736,148.000000,114.000000,78.000000,68.000000,396.000000,14284.032727,3045.566429,1.0,0.408558,4.955760,234.448247,0.345342,0.006615,0.011600,0.210598,0.115837,0.013772,6.279493,32453.500000
max,34472.035440,1125.000000,1004.000000,1008.000000,822.000000,420.000000,64076.600000,13364.210714,1.0,0.467748,748.533642,20210.408330,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,28.521985,43271.000000
